In [1]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/internationaltrade/datasets/internationaltradeinservicesreferencetables/alltables2016/internationaltradeinservices2016.xls'
    inputFile = sourceFolder / 'internationaltradeinservices2016.xlsb'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)    

In [2]:
tab = loadxlstabs(inputFile, sheetids='Table C6 2009-2012')[0]

Loading in\internationaltradeinservices2016.xlsb which has size 750080 bytes
Table names: ['Table C6 2009-2012']


In [3]:
observations = tab.excel_ref('E6').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
Year = tab.excel_ref('E4').expand(RIGHT).is_not_whitespace()

In [5]:
Flow = tab.excel_ref('E3').expand(RIGHT).is_not_blank()

In [6]:
geo1 = tab.excel_ref('A5').expand(DOWN).is_not_blank() - tab.excel_ref('A94').expand(DOWN)
geo1

{<A70 'Royalties and Licenses'>, <A41 'Construction Services'>, <A85 'Other Trade  in Services'>, <A93 '..  Denotes disclosive data'>, <A5 'Agricultural, Mining and On-site Processing Services '>, <A78 'Technical services'>, <A31 'Communication, computer and Information Services'>, <A92 '-   Denotes nil or less than £500,000'>, <A51 'Insurance Services'>, <A47 'Financial Services'>, <A13 'Business and Professional Services'>, <A56 'Merchanting and Other Trade related Services'>, <A62 'Personal, Cultural and Recreational  Services'>, <A89 'Total International Trade in Services'>}

In [7]:
geo3 = tab.excel_ref('C5').expand(DOWN).is_not_blank() - tab.excel_ref('A94').expand(DOWN)
geo3

{<C79 'Architectural'>, <C58 'Other trade - related services'>, <C7 'Mining'>, <C19 'Other business management'>, <C36 'Publishing services'>, <C64 'Health Services'>, <C63 'Audio- Visual and related services '>, <C16 'Management consulting'>, <C27 'Other business and professional services'>, <C81 'Surveying'>, <C48 'Financial'>, <C73 'Purchases and sales of franchises and similar rights'>, <C32 'Postal and courier'>, <C17 'Public relations services'>, <C34 'Computers'>, <C43 'Construction outside the UK'>, <C72 'Other royalties and license fees'>, <C22 'Operational leasing services'>, <C21 'Market research and public opinion polling'>, <C25 'Research and development'>, <C9 'Other on-site processing services'>, <C57 'Merchanting'>, <C8 'Waste treatment and depollution'>, <C42 'Construction in the UK'>, <C26 'Services between related enterprises'>, <C71 'Use of franchise and similar rights fees'>, <C52 'Insurance Claims'>, <C53 'Insurance Premiums'>, <C20 'Legal services'>, <C6 'Agricul

In [8]:
Dimensions = [
            HDim(Year,'Year',DIRECTLY,ABOVE),
            HDim(geo1,'geo1',CLOSEST,ABOVE),
#             HDim(geo2,'geo2',DIRECTLY,LEFT),
            HDim(geo3,'geo3',DIRECTLY,LEFT),
            HDim(Flow, 'Flow',CLOSEST,LEFT),
            HDimConst('Measure Type', 'GBP Total'),
            HDimConst('Unit','gbp-million')
            ]

In [9]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [10]:
new_table = c1.topandas()

In [11]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [12]:
new_table['Year'] = new_table['Year'].astype(int)

In [13]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [14]:
new_table['geo3'].fillna('Total', inplace = True)

In [15]:
new_table['BOP Service'] = new_table['geo1'].fillna('') + '-' + new_table['geo3'].fillna('')

In [16]:
new_table['Flow'] = new_table['Flow'].str.lower()

In [17]:
new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('balances', 'balance'))

In [18]:
new_table = new_table[new_table['Value'] != '' ]

In [19]:
new_table['International Trade Basis'] = 'BOP'

In [20]:
new_table['ONS Partner Geography'] = 'Whole world'

In [21]:
new_table = new_table[['ONS Partner Geography', 'Year','Flow','BOP Service', 'International Trade Basis', 'Measure Type','Value','Unit' ]]

In [22]:
# if is_interactive():
#     SubstancetinationFolder = Path('out')
#     SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
#     new_table.to_csv(SubstancetinationFolder / ('tablec61.csv'), index = False)

In [23]:
new_table.head()

,ONS Partner Geography,Year,Flow,BOP Service,International Trade Basis,Measure Type,Value,Unit
0,Whole world,2009,exports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,10.033,gbp-million
8,Whole world,2009,balance,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,9.693,gbp-million
12,Whole world,2009,exports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,2.456,gbp-million
16,Whole world,2009,imports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,6.461,gbp-million
18,Whole world,2011,imports,"Agricultural, Mining and On-site Processing Se...",BOP,GBP Total,1.73,gbp-million
